In [26]:
import sys
import numpy as np
import matplotlib.pyplot as plt
# get_ipython().magic(u'matplotlib inline')
from pybloom import BloomFilter
import inspect
import matplotlib.font_manager
from sklearn import svm
import os
import glob
import pickle as pkl
import time
# get_ipython().magic(u'matplotlib notebook')
from itertools import product
from joblib import Parallel, delayed
import multiprocessing
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import pickle
from sklearn import mixture
import math
import epsilon_occ as ep

def readFileByLines(filename):
    f = open(filename, 'r')
    lst = f.readlines()
    f.close()
#     print(lst)
#     lst = map(str.rstrip, lst)
    return lst

def parseVdisk(lst):
    lst = [s.split(':') for s in lst]
#     vid = []
#     blc = [] 
    mat = []
    for ele in lst:
#         vid.append(int(ele[1]))
#         blc.append(int(ele[2], 16))
        mat.append([float(int(ele[1]))/1e0,float(int(ele[2], 16))/1e0])
#     print(vid, blc)
#     return (vid, blc)
    return mat

def sumsizeofbloom(bloomfilters):
    agg =0
    for f in bloomfilters.values():
        agg += f.num_bits
    agg = agg/8
    return agg


def sumsizeofmodelsepsilonReal(models):
    agg =0
    for sstableID in models:
        tempmodel, reqdim= models[str(sstableID)]
        # tempsize = 8*(tempmodel.support_vectors_.shape[0]*tempmodel.support_vectors_.shape[1])+ 8*(tempmodel.dual_coef_.shape[0]*tempmodel.dual_coef_.shape[1]) + 4*(tempmodel.support_.shape[0])
        tempsize = tempmodel.get_size()/8.0
        agg += tempsize
        # print(str(tempsize))
        # print(sys.getsizeof(p))
        print(sstableID, tempsize)
    return agg
# In[12]:


# def persistModel(models, D, epsilon, agg_thr, box_thr):
#     pickle_out = open("svmModels/svmmodelstandardscaled_"+namePrefix+str(D)+"_"+str(epsilon)+"_"+str(agg_thr)+"_"+str(box_thr),"wb")
#     pkl.dump(models, pickle_out)
#     pickle_out.close()

def persistModel(models, D, epsilon, agg_thr, box_thr):
    pickle_out = open("svmModels/svmmodelstandardscaled_"+namePrefix+str(D)+"_"+str(epsilon)+"_"+str(agg_thr)+"_"+str(box_thr),"wb")
    pkl.dump(models, pickle_out)
    pickle_out.close()


# In[13]:


def loadModel(D, epsilon, agg_thr, box_thr):
    pickle_out = open("svmModels/svmmodelstandardscaled_"+namePrefixOri+str(D)+"_"+str(epsilon)+"_"+str(agg_thr)+"_"+str(box_thr),"rb")
    models = pkl.load(pickle_out)
    pickle_out.close()
    return models


# In[14]:


def persistBloomFilters(bloomfilters, D, epsilon, agg_thr, box_thr):
    pickle_out = open("bloomfilters/bloomfilters_"+namePrefix+str(D)+"_"+str(epsilon)+"_"+str(agg_thr)+"_"+str(box_thr),"wb")
    pkl.dump(bloomfilters, pickle_out)
    pickle_out.close()


def loadBloomFilters(D, epsilon, agg_thr, box_thr):
    pickle_out = open("bloomfilters/bloomfilters_"+namePrefix+str(D)+"_"+str(epsilon)+"_"+str(agg_thr)+"_"+str(box_thr),"rb")
    bloomfilters = pkl.load(pickle_out)
    pickle_out.close()
    return bloomfilters

# In[15]:


def persistObservations(obs, D, epsilon, agg_thr, box_thr):
    print(obs, D, epsilon, agg_thr, box_thr)
    pickle_out = open("Observations/observations_"+namePrefix+str(D)+"_"+str(epsilon)+"_"+str(agg_thr)+"_"+str(box_thr),"wb")
    pkl.dump(obs, pickle_out)
    pickle_out.close()


def loadObservations(D, epsilon, agg_thr, box_thr):
    pickle_out = open("Observations/observations_"+namePrefix+str(D)+"_"+str(epsilon)+"_"+str(agg_thr)+"_"+str(box_thr),"rb")
    obs = pkl.load(pickle_out)
    pickle_out.close()
    return obs

# In[16]:


def persistTimes(obs):
    pickle_out = open("times/times_"+namePrefix,"wb")
    pkl.dump(obs, pickle_out)
    pickle_out.close()

def loadTimes():
    pickle_out = open("times/times_"+namePrefix,"rb")
    obs = pkl.load(pickle_out)
    pickle_out.close()
    return obs

def getData():
    foldername = "metadata_maps/vdisk/*.db"
    files = sorted(glob.glob(foldername))
    ctr=0
    alltemptableData = []
    for fle in files:
        thisline = readFileByLines(fle)
    #     if(len(thisline)>1000):
    #         continue
        mat = parseVdisk(thisline)
        ssTableData[ctr] = np.array(mat).reshape(-1,2)
        ctr+=1
        alltemptableData+=mat
        print(ctr,len(thisline))
    # print(lines)
    alltemptableData = np.array(alltemptableData).reshape(-1,2)
    return (ssTableData, alltemptableData)

def getScalers():
    scalers = {}
    for i in range(0,len(ssTableData)):
        thistableData = ssTableData[i]
        scaler = preprocessing.StandardScaler().fit(thistableData)
        scalers[i] = scaler
    return scalers

def ngramMaxDimKnown(Xd, gramsize, maxlen):
    X = map(str, Xd)
    numele = len(X)
    reqdim = maxlen - gramsize +1
    
    Y = []
    for i in X:
        tempy = []
        paddedi = i + "0"*(maxlen-len(i))
        for j in range(0,len(paddedi)-gramsize+1):
            tempy.append(paddedi[j:j+gramsize])
        if len(tempy) == 0:
            tempy = [paddedi]
        templeny = len(tempy)
        
        Y.append(tempy)
    return Y

def createNgramsPerSStable(thisssTableData, alltemptableData, mingram, maxgram):
    # mingram = 6
    # maxgram = 6
    initialDimension = alltemptableData.shape[1]
    
    extrapolatedData = thisssTableData
    findimlist = []
    for realFeature in range(initialDimension):
    #     not taking care of negative features
        thisFeatureData = thisssTableData[:,realFeature]
        maxlen = len(str(np.max(alltemptableData[:,realFeature])))
        dimlist = []
        for gramsize in range(mingram, min(maxgram,maxlen)+1):
#             print(thisFeatureData[0], gramsize, maxlen)
#             print(gramsize)
            thisFeatureDataOfGramSize = np.array(ngramMaxDimKnown(thisFeatureData, gramsize, maxlen),dtype=np.float)
#             print(thisFeatureDataOfGramSize[0])
#             print(thisFeatureDataOfGramSize.shape)
            reqdimForLater = thisFeatureDataOfGramSize.shape[1]
#             print(reqdimForLater)
            dimlist.append(reqdimForLater)
            extrapolatedData = np.append(extrapolatedData, thisFeatureDataOfGramSize, axis=1)
        findimlist += dimlist
    return (extrapolatedData, findimlist)
def createNgrams(ssTableData, alltemptableData, mingram, maxgram):
    ngramData = {}
    dimlists = []
    for i in range(len(ssTableData)):
        (bigData, dimlist)=createNgramsPerSStable(ssTableData[i], alltemptableData, mingram, maxgram)
        ngramData[i] = bigData
        dimlists.append(dimlist)
    return (ngramData, dimlists)


gramsize = -1
maxiter = 100
# models = {} # storing models based on sstable ids
# nu = 0.1
# gamma = 1
removeClassifier = False


# In[19]:


# thres = -0.1
# bloomfilters = {}
capacityBloom = 30000
errorRate = 0.1
skipFactor = 50


# In[20]:


ssTableData = {}
separator = ":"
namePrefix = "6nu6gamma_epsilon"
# namePrefixOri = "6nu6gamma_ex4"
namePrefixOri = namePrefix

(ssTableData, alltemptableData) = getData()
ssTableData, dimlist = createNgrams(ssTableData, alltemptableData,4,4)
scalers = getScalers()
# nus = np.logspace(-3,0,6)
# gammas = np.logspace(-3,3,6)
# nus = np.logspace(-3,0,6)
# gammas = np.logspace(-1,3,6)
# num_cores = multiprocessing.cpu_count()

# tempresults = ensemble(6,1e-2,0.9,1e-2)
# print(tempresults)
Ds = map(int,np.logspace(math.log10(2),math.log10(256),10))
epsilons = np.logspace(-4,-1,10)
agg_thrs = np.linspace(0.8,1.0,3)
box_thrs = np.logspace(-4,-1,10)
# results = Parallel(n_jobs=num_cores)(delayed(ensemble)(i,j,k,l) for i,j,k,l in tqdm(product(Ds, epsilons, agg_thrs, box_thrs)))
# print(results)
mshape = (10,10,3,10)

obsdic = {}
fprates = np.zeros(mshape)
fprateslist = []
bfsizelist = []
modelSizeList = []
fpdic = {}
for i,j,k,l in product(Ds, epsilons, agg_thrs, box_thrs):
    print(i,j,k,l)
    try:
        obs = loadObservations(i,j,k,l)
        fprateslist.append(obs[1])
        fpdic[(i,j)] = obs[1]
        print(obs[1])
    except IOError:
        obs = None
        fprateslist.append(1)
        fpdic[(i,j)] = None
        print(obs)
    try:
        bloomfilters = loadBloomFilters(i,j,k,l)
        tempsize = sumsizeofbloom(bloomfilters)
        bfsizelist.append(tempsize)
        print(tempsize)
    except IOError:
        tempsize = None
        bfsizelist.append(1e6)
        print(tempsize)
#     try:
#         models = loadModel(i,j,k,l)
#         tempsize = sumsizeofmodelsepsilonReal(models)
#         modelSizeList.append(tempsize)
#         print(tempsize)
#     except IOError:
#         tempsize = None
#         modelSizeList.append(1e10)
#         print(tempsize)
    
#     fprateslist.append(obs[1])
#     fpdic[(i,j)] = obs[1]
    obsdic[(i,j,k,l)] = obs
fprateslist = np.array(fprateslist).reshape(mshape)
bfsizelist = np.array(bfsizelist).reshape(mshape)
modelSizeList = np.array(modelSizeList).reshape(mshape)
totalSizeList = bfsizelist + modelSizeList




(1, 120691)
(2, 126781)
(3, 125351)
(4, 119538)
(5, 126719)
(6, 25367)
(7, 29100)
(8, 28898)
(9, 30806)
(10, 16550)
(11, 35023)
(12, 5815)
(13, 17015)
(14, 405)
(15, 4266)
(16, 1265)
(17, 16967)
(18, 1963)
(19, 880)
(20, 397)
(21, 1283)
(22, 845)
(23, 368)
(24, 3071)
(2, 0.0001, 0.8, 0.0001)
0.890933321218
55237
(2, 0.0001, 0.8, 0.00021544346900318845)
0.881740947708
73070
(2, 0.0001, 0.8, 0.00046415888336127773)
0.871564217891
94465
(2, 0.0001, 0.8, 0.001)
0.855708509382
120806
(2, 0.0001, 0.8, 0.002154434690031882)
0.835415625521
150697
(2, 0.0001, 0.8, 0.004641588833612777)
0.811033877001
181662
(2, 0.0001, 0.8, 0.01)
0.782411824391
213357
(2, 0.0001, 0.8, 0.021544346900318822)
0.764057365257
239710
(2, 0.0001, 0.8, 0.046415888336127774)
0.746217800191
261366
(2, 0.0001, 0.8, 0.1)
0.730407523511
280546
(2, 0.0001, 0.9, 0.0001)
0.890933321218
55237
(2, 0.0001, 0.9, 0.00021544346900318845)
0.881740947708
73070
(2, 0.0001, 0.9, 0.00046415888336127773)
0.871564217891
94465
(2, 0.0001, 0

1005675
(2, 0.002154434690031882, 1.0, 0.00046415888336127773)
0.0602274620266
1005675
(2, 0.002154434690031882, 1.0, 0.001)
0.0602274620266
1005675
(2, 0.002154434690031882, 1.0, 0.002154434690031882)
0.0602274620266
1005675
(2, 0.002154434690031882, 1.0, 0.004641588833612777)
0.0602274620266
1005675
(2, 0.002154434690031882, 1.0, 0.01)
0.0602274620266
1005675
(2, 0.002154434690031882, 1.0, 0.021544346900318822)
0.0602274620266
1005675
(2, 0.002154434690031882, 1.0, 0.046415888336127774)
0.0602274620266
1005675
(2, 0.002154434690031882, 1.0, 0.1)
0.0602274620266
1005675
(2, 0.004641588833612777, 0.8, 0.0001)
0.980964063423
659
(2, 0.004641588833612777, 0.8, 0.00021544346900318845)
0.980994351309
827
(2, 0.004641588833612777, 0.8, 0.00046415888336127773)
0.980918631593
1048
(2, 0.004641588833612777, 0.8, 0.001)
0.980812623991
1467
(2, 0.004641588833612777, 0.8, 0.002154434690031882)
0.980449169355
1992
(2, 0.004641588833612777, 0.8, 0.004641588833612777)
0.979858555571
2772
(2, 0.00464

81
(2, 0.1, 0.8, 0.1)
0.990383596081
81
(2, 0.1, 0.9, 0.0001)
0.991413384217
36
(2, 0.1, 0.9, 0.00021544346900318845)
0.991413384217
36
(2, 0.1, 0.9, 0.00046415888336127773)
0.991413384217
37
(2, 0.1, 0.9, 0.001)
0.991413384217
40
(2, 0.1, 0.9, 0.002154434690031882)
0.991413384217
43
(2, 0.1, 0.9, 0.004641588833612777)
0.99147395999
68
(2, 0.1, 0.9, 0.01)
0.991292232672
78
(2, 0.1, 0.9, 0.021544346900318822)
0.990383596081
81
(2, 0.1, 0.9, 0.046415888336127774)
0.990383596081
81
(2, 0.1, 0.9, 0.1)
0.990383596081
81
(2, 0.1, 1.0, 0.0001)
0.0602274620266
1005675
(2, 0.1, 1.0, 0.00021544346900318845)
0.0602274620266
1005675
(2, 0.1, 1.0, 0.00046415888336127773)
0.0602274620266
1005675
(2, 0.1, 1.0, 0.001)
0.0602274620266
1005675
(2, 0.1, 1.0, 0.002154434690031882)
0.0602274620266
1005675
(2, 0.1, 1.0, 0.004641588833612777)
0.0602274620266
1005675
(2, 0.1, 1.0, 0.01)
0.0602274620266
1005675
(2, 0.1, 1.0, 0.021544346900318822)
0.0602274620266
1005675
(2, 0.1, 1.0, 0.046415888336127774)
0.06

1005675
(3, 0.001, 1.0, 0.1)
0.0602274620266
1005675
(3, 0.002154434690031882, 0.8, 0.0001)
0.969303227174
2180
(3, 0.002154434690031882, 0.8, 0.00021544346900318845)
0.968258295095
3135
(3, 0.002154434690031882, 0.8, 0.00046415888336127773)
0.967697969197
4684
(3, 0.002154434690031882, 0.8, 0.001)
0.966728756834
6588
(3, 0.002154434690031882, 0.8, 0.002154434690031882)
0.965532385322
8444
(3, 0.002154434690031882, 0.8, 0.004641588833612777)
0.963942271289
10042
(3, 0.002154434690031882, 0.8, 0.01)
0.963169930186
10961
(3, 0.002154434690031882, 0.8, 0.021544346900318822)
0.962352157255
12171
(3, 0.002154434690031882, 0.8, 0.046415888336127774)
0.961488952493
13078
(3, 0.002154434690031882, 0.8, 0.1)
0.9611406418
13460
(3, 0.002154434690031882, 0.9, 0.0001)
0.969303227174
2180
(3, 0.002154434690031882, 0.9, 0.00021544346900318845)
0.968258295095
3135
(3, 0.002154434690031882, 0.9, 0.00046415888336127773)
0.967697969197
4684
(3, 0.002154434690031882, 0.9, 0.001)
0.966728756834
6588
(3, 0

1005675
(3, 0.046415888336127774, 1.0, 0.002154434690031882)
0.0602274620266
1005675
(3, 0.046415888336127774, 1.0, 0.004641588833612777)
0.0602274620266
1005675
(3, 0.046415888336127774, 1.0, 0.01)
0.0602274620266
1005675
(3, 0.046415888336127774, 1.0, 0.021544346900318822)
0.0602274620266
1005675
(3, 0.046415888336127774, 1.0, 0.046415888336127774)
0.0602274620266
1005675
(3, 0.046415888336127774, 1.0, 0.1)
0.0602274620266
1005675
(3, 0.1, 0.8, 0.0001)
0.98680962549
37
(3, 0.1, 0.8, 0.00021544346900318845)
0.98680962549
37
(3, 0.1, 0.8, 0.00046415888336127773)
0.98680962549
38
(3, 0.1, 0.8, 0.001)
0.986794481547
41
(3, 0.1, 0.8, 0.002154434690031882)
0.986794481547
44
(3, 0.1, 0.8, 0.004641588833612777)
0.986688473945
70
(3, 0.1, 0.8, 0.01)
0.985870701013
1646
(3, 0.1, 0.8, 0.021544346900318822)
0.985052928081
2310
(3, 0.1, 0.8, 0.046415888336127774)
0.985052928081
2310
(3, 0.1, 0.8, 0.1)
0.985052928081
2310
(3, 0.1, 0.9, 0.0001)
0.98680962549
37
(3, 0.1, 0.9, 0.00021544346900318845)

...

1005675
(149, 0.004641588833612777, 1.0, 0.046415888336127774)
0.0602274620266
1005675
(149, 0.004641588833612777, 1.0, 0.1)
0.0602274620266
1005675
(149, 0.01, 0.8, 0.0001)
0.985052928081
87
(149, 0.01, 0.8, 0.00021544346900318845)
0.985068072025
88
(149, 0.01, 0.8, 0.00046415888336127773)
0.985037784138
92
(149, 0.01, 0.8, 0.001)
0.984992352309
122
(149, 0.01, 0.8, 0.002154434690031882)
0.984871200763
156
(149, 0.01, 0.8, 0.004641588833612777)
0.984538034013
765
(149, 0.01, 0.8, 0.01)
0.983659685309
2195
(149, 0.01, 0.8, 0.021544346900318822)
0.983356806445
2535
(149, 0.01, 0.8, 0.046415888336127774)
0.983084215468
2650
(149, 0.01, 0.8, 0.1)
0.982342162252
2793
(149, 0.01, 0.9, 0.0001)
0.980222010207
156
(149, 0.01, 0.9, 0.00021544346900318845)
0.980191722321
181
(149, 0.01, 0.9, 0.00046415888336127773)
0.980191722321
239
(149, 0.01, 0.9, 0.001)
0.980085714718
358
(149, 0.01, 0.9, 0.002154434690031882)
0.979722260082
445
(149, 0.01, 0.9, 0.004641588833612777)
0.979555676707
1114
(149

1005675
(256, 0.0001, 1.0, 0.021544346900318822)
0.0602274620266
1005675
(256, 0.0001, 1.0, 0.046415888336127774)
0.0602274620266
1005675
(256, 0.0001, 1.0, 0.1)
0.0602274620266
1005675
(256, 0.00021544346900318845, 0.8, 0.0001)
0.949631244984
13277
(256, 0.00021544346900318845, 0.8, 0.00021544346900318845)
0.94804113095
17956
(256, 0.00021544346900318845, 0.8, 0.00046415888336127773)
0.946541880575
23437
(256, 0.00021544346900318845, 0.8, 0.001)
0.943558523768
33376
(256, 0.00021544346900318845, 0.8, 0.002154434690031882)
0.939030484758
38246
(256, 0.00021544346900318845, 0.8, 0.004641588833612777)
0.93489618827
43330
(256, 0.00021544346900318845, 0.8, 0.01)
0.931064770645
46419
(256, 0.00021544346900318845, 0.8, 0.021544346900318822)
0.925385791953
49382
(256, 0.00021544346900318845, 0.8, 0.046415888336127774)
0.921296927294
52151
(256, 0.00021544346900318845, 0.8, 0.1)
0.917722956703
54770
(256, 0.00021544346900318845, 0.9, 0.0001)
0.929928974906
26075
(256, 0.00021544346900318845, 

4913
(256, 0.004641588833612777, 0.9, 0.0001)
0.975921130344
361
(256, 0.004641588833612777, 0.9, 0.00021544346900318845)
0.975981706117
489
(256, 0.004641588833612777, 0.9, 0.00046415888336127773)
0.975921130344
717
(256, 0.004641588833612777, 0.9, 0.001)
0.975693971196
1199
(256, 0.004641588833612777, 0.9, 0.002154434690031882)
0.975512243878
1894
(256, 0.004641588833612777, 0.9, 0.004641588833612777)
0.974770190662
3488
(256, 0.004641588833612777, 0.9, 0.01)
0.974088713219
4688
(256, 0.004641588833612777, 0.9, 0.021544346900318822)
0.973604107037
5662
(256, 0.004641588833612777, 0.9, 0.046415888336127774)
0.972968061424
6201
(256, 0.004641588833612777, 0.9, 0.1)
0.972407735526
6532
(256, 0.004641588833612777, 1.0, 0.0001)
0.0602274620266
1005675
(256, 0.004641588833612777, 1.0, 0.00021544346900318845)
0.0602274620266
1005675
(256, 0.004641588833612777, 1.0, 0.00046415888336127773)
0.0602274620266
1005675
(256, 0.004641588833612777, 1.0, 0.001)
0.0602274620266
1005675
(256, 0.0046415

1005675
(256, 0.1, 1.0, 0.01)
0.0602274620266
1005675
(256, 0.1, 1.0, 0.021544346900318822)
0.0602274620266
1005675
(256, 0.1, 1.0, 0.046415888336127774)
0.0602274620266
1005675
(256, 0.1, 1.0, 0.1)
0.0602274620266
1005675


ValueError: cannot reshape array of size 0 into shape (10,10,3,10)

In [27]:
minval = 1.0
for i in obsdic:
    print(i, obsdic[i][1])
    if obsdic[i][1] < minval:
        minval = obsdic[i][1]
        minpair =i


((3, 0.001, 0.9, 0.1), 0.9478442596883376)
((256, 0.00021544346900318845, 0.9, 0.01), 0.851831659927612)
((17, 0.0001, 1.0, 0.1), 0.06022746202656248)
((3, 0.00021544346900318845, 1.0, 0.002154434690031882), 0.06022746202656248)
((5, 0.00021544346900318845, 0.9, 0.001), 0.8661123983462814)
((29, 0.0001, 0.8, 0.046415888336127774), 0.7065558130025896)
((149, 0.00021544346900318845, 1.0, 0.00046415888336127773), 0.06022746202656248)
((2, 0.0001, 0.9, 0.00021544346900318845), 0.8817409477079642)
((149, 0.00046415888336127773, 0.8, 0.00046415888336127773), 0.9618069753002286)
((149, 0.004641588833612777, 0.8, 0.046415888336127774), 0.9777232595823301)
((3, 0.004641588833612777, 0.8, 0.001), 0.9758605545711992)
((17, 0.002154434690031882, 1.0, 0.01), 0.06022746202656248)
((17, 0.00046415888336127773, 1.0, 0.021544346900318822), 0.06022746202656248)
((50, 0.021544346900318822, 0.8, 0.002154434690031882), 0.9856435418654309)
((3, 0.00021544346900318845, 0.8, 0.001), 0.8987475958990202)
((29, 

((3, 0.00046415888336127773, 0.9, 0.0001), 0.9438311147456575)
((50, 0.004641588833612777, 1.0, 0.0001), 0.06022746202656248)
((149, 0.0001, 1.0, 0.004641588833612777), 0.06022746202656248)
((5, 0.00021544346900318845, 0.8, 0.001), 0.8661123983462814)
((2, 0.01, 1.0, 0.046415888336127774), 0.06022746202656248)
((10, 0.00046415888336127773, 0.9, 0.0001), 0.9061681280571835)
((3, 0.021544346900318822, 1.0, 0.001), 0.06022746202656248)
((2, 0.0001, 0.9, 0.0001), 0.8909333212181788)
((149, 0.00021544346900318845, 0.9, 0.01), 0.8454560598488634)
((149, 0.1, 0.9, 0.046415888336127774), 0.9849772083655142)
((87, 0.001, 0.9, 0.1), 0.9513425105629004)
((2, 0.021544346900318822, 0.9, 0.1), 0.9846440416155559)
((50, 0.1, 0.8, 0.0001), 0.9875062468765617)
((3, 0.046415888336127774, 1.0, 0.1), 0.06022746202656248)
((17, 0.004641588833612777, 0.8, 0.0001), 0.9808126239910347)
((87, 0.00046415888336127773, 0.9, 0.021544346900318822), 0.9253555040661487)
((87, 0.001, 0.9, 0.00021544346900318845), 0.96

In [28]:
print(minval)
print(minpair)

0.0602274620266
(17, 0.0001, 1.0, 0.1)


In [3]:
tempmodel = loadModel(Ds[0],epsilons[0], agg_thrs[0],box_thrs[0])

In [10]:
tempmodel['0'][0].get_size()

AttributeError: 'list' object has no attribute 'shape'

In [18]:
len(tempmodel['0'][0].intervals)

2

In [19]:
np.array(tempmodel['0'][0].intervals).flatten()

array([list([(-2.651980400699701, -2.647645085800939), (-2.5983813090279457, -2.59204575999297), (-2.5705989774597975, -2.5646775040396865), (-2.541779970693748, -2.5332737087946056), (-2.5265636050821465, -2.519632089730062), (-2.511790863845021, -2.506296051497873), (-2.497982292847528, -2.4900597875709387), (-2.4696735925932316, -2.461443863120458), (-2.4383418760326436, -2.4334341460286693), (-2.413873995818004, -2.410557437500271), (-2.409709043832691, -2.4050397105622663), (-2.399630736990676, -2.3926048069448305), (-2.391798502546349, -2.3883887317190444), (-2.3821454812205554, -2.378062053265533), (-2.3767867332250945, -2.3729079951607974), (-2.37079837820751, -2.3593593853011767), (-2.3551455371359222, -2.350632836828431), (-2.341913156075188, -2.3336247488213595), (-2.3323330883482543, -2.3014544901442675), (-2.2994414586691634, -2.2961991860443227), (-2.295308231486192, -2.2873363136948193), (-2.283370205448706, -2.2741725271899536), (-2.273317226871661, -2.270075744917818),